In [1]:
#!pip install matlabengine
import matlab.engine

In [2]:
eng = matlab.engine.start_matlab()

In [3]:
eng.cd(r'C:\Users\Budde\Downloads\matstuff',nargout=0)

In [4]:
import pde
import numpy as np
import scipy as sp
import numpy as np
from numpy import linalg
from scipy.optimize import minimize
from scipy.optimize import least_squares
import sklearn
from sklearn.decomposition import KernelPCA
import time
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import scipy.sparse as sparse
import scipy.sparse.linalg as linalg
from numpy.random import multivariate_normal
from numpy.linalg import eig
import warnings
warnings.filterwarnings("ignore")
import sympy as sym
from sklearn.utils.extmath import randomized_svd
import numpy as np

from pde import CartesianGrid, solve_laplace_equation


In [5]:
def POD_Train(f,g,M):
    f1 = np.shape(f)[0]
    f2 = np.shape(f)[1]
    fvals = []
    gvals = []
    f = f.copy()
    g = g.copy()
    easymeans1 = np.mean(f,axis=1)
    easymeans2 = np.mean(g,axis=1)
    easymeans1 = easymeans1.reshape(f1,1)
    easymeans2 = easymeans2.reshape(np.shape(g)[0],1)
    for i in range(np.shape(f)[0]):
        f[:][i] = f[:][i] - easymeans1[i]
    for i in range(np.shape(g)[0]):
        g[:][i] = g[:][i] - easymeans2[i]
    W = np.concatenate((f,g))
    RSV_computed = []
    U,W_svd,V = np.linalg.svd(W,compute_uv=True)
    for i in range(M):
        x = V[:,i]
        RSV_computed.append(np.dot(W,x))
    RSV_computed = np.array(RSV_computed).T
    for i in range(M):
        fvals.append(RSV_computed[:f1,i])
        gvals.append(RSV_computed[f1:,i])
    fvals,gvals = np.array(fvals),np.array(gvals)
    return fvals.T,gvals.T,easymeans1,easymeans2
def PODMM_Predict(g_test,zeta_f,zeta_g,f_bar,g_bar,M):
    g_test = g_test.copy()
    f_bar = f_bar.copy()
    zeta_f = zeta_f.copy()
    zeta_g = zeta_g.copy()
    f_bar = f_bar.copy()
    g_bar = g_bar.copy()
    alpha_PODMM = []
    g_test = g_test.reshape(-1,1)
    #print(g_test.shape,zeta_f.shape,zeta_g.shape,f_bar.shape,g_bar.shape,)
    objective_func = lambda y:(g_test-(g_bar + np.dot(zeta_g,y.T).reshape(-1,1))).flatten()
    y0 = np.random.random(M)
    gam = least_squares(objective_func,y0)
    alph = gam.x
    return f_bar + np.dot(zeta_f,alph.T).reshape(-1,1)
def ADRSource(Lx, Nx, Source, omega, v, kappa):
    """
    Solve the advection-diffusion-reaction equation
    input:
    Lx: float, the right end of x
    Nx: int, nunber of x
    Source: 1d array of size Nx
    omega: 1d array of size Nx
    v: 1d array of size Nx+1
    kappa: 1d array of size Nx
    return:
    Solution: 1d array of size Nx
    Q: float, quantiy of interest
    """
    Source = np.full((Nx),Source)
    omega = np.full((Nx),omega)
    v = np.full((Nx),v)
    kappa = np.full((Nx),kappa)
    A = sparse.dia_matrix((Nx,Nx))   
    dx = Lx/(Nx-1)
    i2dx2 = 1.0/(dx*dx)
    #fill diagonal of A
    A.setdiag(2*i2dx2*omega + np.sign(v)*v/dx + kappa)
    #fill off diagonals of A
    A.setdiag(-i2dx2*omega[1:Nx] + 0.5*(1-np.sign(v[1:Nx]))*v[1:Nx]/dx,1)
    A.setdiag(-i2dx2*omega[0:(Nx-1)] - 0.5*(np.sign(v[0:(Nx-1)])+1)*v[0:(Nx-1)]/dx,-1)
    #solve A x = Source
    Solution = linalg.spsolve(A,Source)
    # Trapezoid rule
    Q = np.sum(Solution[1:-1]*kappa[1:-1]*dx) + \
        Solution[0]*kappa[0]*dx/2 + Solution[-1]*kappa[-1]*dx/2
    return Solution, Q
def lazy(mean,std):
    A = [[1,-1],[1,1]]
    e1 = np.sqrt(12)*std
    e2 = 2*mean
    b = [e1,e2]
    return np.linalg.solve(A,b)


In [6]:
N = 20000 #points at fine resolution
M = 100 # points at coarse resolution
J = 200 # number of solutions

In [7]:
v_a_b = lazy(10,1)
omega_a_b = lazy(20,2)
k_l_a_b = lazy(2,.2)
k_h_a_b = lazy(.1,.01)
q_a_b = lazy(1,.1)
v_sample = np.random.uniform(v_a_b[0],v_a_b[1],N)
omega_sample = np.random.uniform(omega_a_b[0],omega_a_b[1],N)
k_l_sample = np.random.uniform(k_l_a_b[0],k_l_a_b[1],N)
k_h_sample = np.random.uniform(k_h_a_b[0],k_h_a_b[1],N)
q_sample = np.random.uniform(q_a_b[0],q_a_b[1],N)

In [8]:
y = np.linspace(0,1,N)
x = np.linspace(0,1,M)
F_soln = np.zeros((N,J))
F_soln_noise = np.zeros((N,J))
for i in range(J):
    F_soln[:,i] = (ADRSource(1,N,q_sample[i]*y*(10-y),omega_sample[i],v_sample[i],k_l_sample[i])[0])
G_soln = np.zeros((M,J))
for i in range(J):
    G_soln[:,i] = (ADRSource(1,M,q_sample[i]*x*(10-x),omega_sample[i],v_sample[i],k_l_sample[i])[0])
#G_soln = G_soln.T
test_g = 2*G_soln[:,1] + 1.5*G_soln[:,2] - G_soln[:,3]
#print(test_g.shape,test_f.shape)
test_f = 2*F_soln[:,1] + 1.5*F_soln[:,2] - F_soln[:,3]
F_soln[:,i].shape

(20000,)

In [11]:
fmeans = np.mean(F_soln,axis=1)
gmeans = np.mean(G_soln,axis=1)

var = 0
for i in range(N):
    var = (.1*fmeans[i])**2
    eps = np.random.multivariate_normal(np.zeros(N),var*np.eye(N))
    F_soln_noise[:,i] = F_soln[:,i] + eps

for i in range(M):
    var = (.1*gmeans[i])**2
    eps = np.random.multivariate_normal(np.zeros(M),var*np.eye(M))
    G_soln_noise[:,i] = G_soln[:,i] + eps

MemoryError: Unable to allocate 2.98 GiB for an array with shape (20000, 20000) and data type float64

In [ ]:
print(fmeans)

In [ ]:
G_soln_noise.shape

In [123]:
F_soln[0:100,:].shape

(100, 200)

In [175]:
pack = eng.svde(F_soln.T,{'AR'})

In [176]:
b = pack.get('AR')

In [177]:
b = np.array(b)

In [178]:
b.shape

(200, 2000)

In [179]:
pack.get('ARmerit')

0.9948535330143653

In [180]:
b

array([[-0.00507809, -0.00496559,  0.00424474, ..., -0.00478503,
         0.00457011, -0.01724948],
       [-0.00508244, -0.00496985,  0.00424838, ..., -0.00478914,
         0.00457403, -0.01726429],
       [-0.00738727, -0.00722362,  0.00617496, ..., -0.00696095,
         0.00664829, -0.02509341],
       ...,
       [-0.00395224, -0.00386468,  0.00330365, ..., -0.00372416,
         0.00355688, -0.01342515],
       [-0.0065466 , -0.00640157,  0.00547226, ..., -0.0061688 ,
         0.00589172, -0.0222378 ],
       [-0.00654671, -0.00640169,  0.00547235, ..., -0.00616891,
         0.00589183, -0.02223819]])

In [181]:
G_soln = []
for i in range(200):
    G_soln.append(ADRSource(1,M,q_sample[i]*x*(10-x),omega_sample[i],v_sample[i],k_l_sample[i])[0])
G_soln = np.array(G_soln)
#G_soln = G_soln.T

In [182]:
fv,gv,fm,gm = POD_Train(b.T,G_soln.T,100)

In [183]:
test_g = 2*(G_soln.T[:,1]) + 1.5*(G_soln.T[:,2]) - G_soln.T[:,3]

In [184]:
(b.T).shape
(test_g).shape

(100,)

In [185]:
newf = PODMM_Predict(test_g,fv,gv,fm,gm,100)

In [186]:
np.linalg.norm(newf-test_f)

857.5435949991277

In [187]:
test_f

array([ 0.44222529,  0.47152554, -0.14804142, ..., -0.7946534 ,
       -0.12720256, -0.27236147])

In [188]:
newf

array([[-0.02912849],
       [-0.02848321],
       [ 0.0243483 ],
       ...,
       [-0.0274475 ],
       [ 0.02621468],
       [-0.09894503]])

In [ ]:
F_soln = []
for i in range(200):
    F_soln.append(ADRSource(1,N,q_sample[i]*y*(10-y),omega_sample[i],v_sample[i],k_l_sample[i])[0] + eps)
F_soln = np.array(F_soln)